In [1]:
import tensorflow as tf
from tensorflow.python.keras import layers, models
import numpy as np

2022-05-18 00:29:51.459334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-18 00:29:51.459358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Loading the dataset
dataset = np.load('character.npy', allow_pickle = True)

# separating images and labels
labels = dataset[:,1]
images = dataset[:,0]

# The different classes of labels, example: 2,3,A,B,C,x,y etc
classes = np.unique(labels)

# Making a dictionary to get a numeric equivalent of all classes
classes = {classes[i]:i for i in range(classes.__len__())}
labels = np.array([classes[i] for i in labels])

In [3]:
def train_test_split(x : np.ndarray, y: np.ndarray):
    """
    Splits the data into 80-20 train-test split.
    """
    def __shuffle__(x,y):
        """
        Shuffles the data
        """
        n = y.shape[0]
        permutation = np.random.permutation(n)
        return x[permutation], y[permutation]
    
    x,y = __shuffle__(x,y)
    n = y.shape[0]
    train_x = x[:int(n*0.8)]
    test_x = x[int(n*0.8):]
    train_y = y[:int(n*0.8)]
    test_y = y[int(n*0.8):]
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = train_test_split(images, labels)

In [4]:
# Converting our numpy data to tensors
train_x = tf.convert_to_tensor([tf.convert_to_tensor(i, np.uint8) for i in train_x])
train_y = tf.convert_to_tensor([tf.convert_to_tensor(i, np.uint8) for i in train_y])
test_x = tf.convert_to_tensor([tf.convert_to_tensor(i, np.uint8) for i in test_x])
test_y = tf.convert_to_tensor([tf.convert_to_tensor(i, np.uint8) for i in test_y])

# The image has all dtype of int,
# but we need float for our calculations
# So, we will cast the tensors to float datatype
train_x = tf.cast(train_x, tf.float64)
test_x = tf.cast(test_x, tf.float64)


2022-05-18 00:29:53.316767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-18 00:29:53.316800: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-18 00:29:53.316824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dino): /proc/driver/nvidia/version does not exist
2022-05-18 00:29:53.317115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 00:29:59.734670: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 876787200 exceeds 10% 

In [5]:
# Our model
# It consists of a Convolutional layer with 5 kernels
# followed by max pooling layer
# followed by another convolutional layer with with 5 kernels
# followed by yes you guessed it, max pooling layer
# Then a flattening layer
# followed by an mlp with 81 neurons in hidden layer
# and finally 51 in the output layer

model = models.Sequential()
model.add(layers.Conv2D(5, (2,2), activation='relu', input_shape = (30,30,1)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(5, (2,2), activation = 'relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(81, activation='relu'))
model.add(layers.Dense(51))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 29, 29, 5)         25        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 5)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 5)         105       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 5)           0         
_________________________________________________________________
flatten (Flatten)            (None, 180)               0         
_________________________________________________________________
dense (Dense)                (None, 81)                14661     
_________________________________________________________________
dense_1 (Dense)              (None, 51)                4

In [7]:
model.compile(optimizer='adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )
model.fit(train_x, train_y, epochs = 15, validation_data = (test_x, test_y))

Epoch 1/15
3806/3806 [==============================] - 31s 8ms/step - loss: 0.6934 - sparse_categorical_accuracy: 0.8364 - val_loss: 0.0035 - val_sparse_categorical_accuracy: 0.9584
Epoch 2/15
3806/3806 [==============================] - 29s 8ms/step - loss: 0.0172 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.0056 - val_sparse_categorical_accuracy: 0.9786
Epoch 3/15
3806/3806 [==============================] - 28s 7ms/step - loss: 0.0110 - sparse_categorical_accuracy: 0.9824 - val_loss: 0.0032 - val_sparse_categorical_accuracy: 0.9851
Epoch 4/15
3806/3806 [==============================] - 29s 7ms/step - loss: 0.0042 - sparse_categorical_accuracy: 0.9871 - val_loss: 0.0023 - val_sparse_categorical_accuracy: 0.9887
Epoch 5/15
3806/3806 [==============================] - 29s 8ms/step - loss: 0.0057 - sparse_categorical_accuracy: 0.9899 - val_loss: 0.0149 - val_sparse_categorical_accuracy: 0.9907
Epoch 6/15
3806/3806 [==============================] - 29s 8ms/step - loss: 0.0051 -

In [8]:
model.save("myModel.h5")